In [1]:
import gzip
import os
import shutil

import requests


def download_and_extract_dataset(url, save_path, folder_path):
    """Download and extract dataset if it doesn't exist."""
    if not os.path.exists(save_path):
        print(f"Downloading {os.path.basename(save_path)}...")
        response = requests.get(url)
        with open(save_path, "wb") as file:
            file.write(response.content)

        decompressed_file_name = os.path.splitext(os.path.basename(save_path))[0]
        decompressed_file_path = os.path.join(folder_path, decompressed_file_name)

        with gzip.open(save_path, "rb") as f_in:
            with open(decompressed_file_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)

        print(f"{decompressed_file_name} downloaded and extracted.")
    else:
        print(f"{os.path.basename(save_path)} already exists.")


file_info = [
    (
        "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
        "train-images-idx3-ubyte.gz",
    ),
    (
        "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
        "train-labels-idx1-ubyte.gz",
    ),
    (
        "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
        "t10k-images-idx3-ubyte.gz",
    ),
    (
        "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz",
        "t10k-labels-idx1-ubyte.gz",
    ),
]

folder_name = "tmp/mnist"
folder_path = os.path.join(os.getcwd(), folder_name)

os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist

# Download and extract each file
for url, file_name in file_info:
    path_to_save = os.path.join(folder_path, file_name)
    download_and_extract_dataset(url, path_to_save, folder_path)

# %%
import numpy as np


def read_idx3_ubyte_image_file(filename):
    """Read IDX3-ubyte formatted image data."""
    with open(filename, "rb") as f:
        magic_num = int.from_bytes(f.read(4), byteorder="big")
        num_images = int.from_bytes(f.read(4), byteorder="big")
        num_rows = int.from_bytes(f.read(4), byteorder="big")
        num_cols = int.from_bytes(f.read(4), byteorder="big")

        if magic_num != 2051:
            raise ValueError(f"Invalid magic number: {magic_num}")

        images = np.zeros((num_images, num_rows, num_cols), dtype=np.uint8)

        for i in range(num_images):
            for r in range(num_rows):
                for c in range(num_cols):
                    pixel = int.from_bytes(f.read(1), byteorder="big")
                    images[i, r, c] = pixel

    return images


def read_idx1_ubyte_label_file(filename):
    """Read IDX1-ubyte formatted label data."""
    with open(filename, "rb") as f:
        magic_num = int.from_bytes(f.read(4), byteorder="big")
        num_labels = int.from_bytes(f.read(4), byteorder="big")

        if magic_num != 2049:
            raise ValueError(f"Invalid magic number: {magic_num}")

        labels = np.zeros(num_labels, dtype=np.uint8)

        for i in range(num_labels):
            labels[i] = int.from_bytes(f.read(1), byteorder="big")

    return labels


# Example usage
folder_path = os.path.join(
    os.getcwd(), folder_name
)  # Adjust this path to where you stored the files

train_images = read_idx3_ubyte_image_file(
    os.path.join(folder_path, "train-images-idx3-ubyte")
)
train_labels = read_idx1_ubyte_label_file(
    os.path.join(folder_path, "train-labels-idx1-ubyte")
)
test_images = read_idx3_ubyte_image_file(
    os.path.join(folder_path, "t10k-images-idx3-ubyte")
)
test_labels = read_idx1_ubyte_label_file(
    os.path.join(folder_path, "t10k-labels-idx1-ubyte")
)

print(
    f"Shape of train_images: {train_images.shape}"
)  # Should output "Shape of train_images: (60000, 28, 28)"
print(
    f"Shape of train_labels: {train_labels.shape}"
)  # Should output "Shape of train_labels: (60000,)"
print(
    f"Shape of test_images: {test_images.shape}"
)  # Should output "Shape of test_images: (10000, 28, 28)"
print(
    f"Shape of test_labels: {test_labels.shape}"
)  # Should output "Shape of test_labels: (10000,)"

# %%
# Reshape the datasets from 3D to 2D
train_images_2d = train_images.reshape(
    train_images.shape[0], -1
)  # -1 infers the size from the remaining dimensions
test_images_2d = test_images.reshape(test_images.shape[0], -1)

train-images-idx3-ubyte.gz already exists.
train-labels-idx1-ubyte.gz already exists.
t10k-images-idx3-ubyte.gz already exists.
t10k-labels-idx1-ubyte.gz already exists.
Shape of train_images: (60000, 28, 28)
Shape of train_labels: (60000,)
Shape of test_images: (10000, 28, 28)
Shape of test_labels: (10000,)


In [35]:
def integral_image(img):
    """
    Compute the integral image of the input img.
    """
    # Convert the image to int32 type
    img = img.astype(np.int32)
    int_img = np.zeros_like(img)

    for x in range(img.shape[1]):
        for y in range(img.shape[0]):
            int_img[y, x] = (
                img[y, x]
                + (int_img[y - 1, x] if y - 1 >= 0 else 0)
                + (int_img[y, x - 1] if x - 1 >= 0 else 0)
                - (int_img[y - 1, x - 1] if x - 1 >= 0 and y - 1 >= 0 else 0)
            )
    return int_img


def haar_feature(int_img, type, x, y, w, h):
    """
    Compute the Haar feature value for a given type, position and size.
    """
    if type == "2h":
        white = (
            int_img[y, x + w]
            - int_img[y, x]
            + int_img[y + h, x + w]
            - int_img[y + h, x]
        )
        black = (
            int_img[y, x + 2 * w]
            - int_img[y, x + w]
            + int_img[y + h, x + 2 * w]
            - int_img[y + h, x + w]
        )
        return black - white

    elif type == "2v":
        white = (
            int_img[y, x]
            - int_img[y, x + w]
            + int_img[y + h // 2, x]
            - int_img[y + h // 2, x + w]
        )
        black = (
            int_img[y + h // 2, x]
            - int_img[y + h // 2, x + w]
            + int_img[y + h, x]
            - int_img[y + h, x + w]
        )
        return black - white

    elif type == "3h":
        white1 = (
            int_img[y, x + w]
            - int_img[y, x]
            + int_img[y + h, x + w]
            - int_img[y + h, x]
        )
        black = (
            int_img[y, x + 2 * w]
            - int_img[y, x + w]
            + int_img[y + h, x + 2 * w]
            - int_img[y + h, x + w]
        )
        white2 = (
            int_img[y, x + 3 * w]
            - int_img[y, x + 2 * w]
            + int_img[y + h, x + 3 * w]
            - int_img[y + h, x + 2 * w]
        )
        return black - (white1 + white2)

    elif type == "3v":
        white1 = (
            int_img[y, x]
            - int_img[y, x + w]
            + int_img[y + h // 3, x]
            - int_img[y + h // 3, x + w]
        )
        black = (
            int_img[y + h // 3, x]
            - int_img[y + h // 3, x + w]
            + int_img[y + 2 * h // 3, x]
            - int_img[y + 2 * h // 3, x + w]
        )
        white2 = (
            int_img[y + 2 * h // 3, x]
            - int_img[y + 2 * h // 3, x + w]
            + int_img[y + h, x]
            - int_img[y + h, x + w]
        )
        return black - (white1 + white2)

    else:
        return 0


def extract_haar_features(img):
    """
    Extracts Haar features from an MNIST image.
    """
    int_img = integral_image(img)
    features = []
    for type in ["2h", "2v", "3h", "3v"]:
        for x in range(0, 28):
            for y in range(0, 28):
                for w in range(1, 14):  # limiting width to avoid out-of-bounds
                    if type == "2h" and x + 2 * w <= 27 and y + 1 <= 27:
                        features.append(haar_feature(int_img, type, x, y, w, 1))
                    elif type == "2v" and y + 2 * w <= 27 and x + w <= 27:
                        features.append(haar_feature(int_img, type, x, y, w, 1))
                    elif type == "3h" and x + 3 * w <= 27 and y + 1 <= 27:
                        features.append(haar_feature(int_img, type, x, y, w, 1))
                    elif type == "3v" and y + 3 * w <= 27 and x + w <= 27:
                        features.append(haar_feature(int_img, type, x, y, w, 1))
    return features


def compact_haar_features(img):
    """Extract Haar features using a 7x7 window for horizontal and vertical edges."""
    int_img = integral_image(img)

    w, h = 7, 7  # Window size
    features = []

    for y in range(0, img.shape[0] - h):
        for x in range(0, img.shape[1] - w):
            # Horizontal edge feature
            white = (
                int_img[y + h // 2, x]
                - int_img[y + h // 2, x + w]
                + int_img[y, x + w]
                - int_img[y, x]
            )
            black = (
                int_img[y + h, x + w]
                - int_img[y + h, x]
                - int_img[y + h // 2, x + w]
                + int_img[y + h // 2, x]
            )
            features.append(black - white)

            # Vertical edge feature
            white = (
                int_img[y, x + w // 2]
                - int_img[y, x]
                + int_img[y + h, x + w // 2]
                - int_img[y + h, x]
            )
            black = (
                int_img[y + h, x + w]
                - int_img[y + h, x + w // 2]
                - int_img[y, x + w]
                + int_img[y, x + w // 2]
            )
            features.append(black - white)

    return features


def compact_haar_features2(img):
    int_img = integral_image(img)
    h, w = 7, 7  # Size of the grid cell
    features = []

    for y in range(0, img.shape[0] - h, h):
        for x in range(0, img.shape[1] - w, w):
            # Horizontal edge feature
            white = (
                int_img[y, x]
                + int_img[y + h // 2, x + w - 1]
                - int_img[y + h // 2, x]
                - int_img[y, x + w - 1]
            )
            black = (
                int_img[y + h // 2, x]
                + int_img[y + h, x + w - 1]
                - int_img[y + h, x]
                - int_img[y + h // 2, x + w - 1]
            )
            features.append(black - white)

            # Vertical edge feature
            white = (
                int_img[y, x]
                + int_img[y + h, x + w // 2 - 1]
                - int_img[y, x + w // 2 - 1]
                - int_img[y + h, x]
            )
            if x + w == img.shape[1]:  # Check if we're at the boundary
                black = (
                    int_img[y, x + w // 2]
                    + int_img[y + h, x + w - 1]
                    - int_img[y + h, x + w - 1]
                    - int_img[y + h, x + w // 2 - 1]
                )
            else:
                black = (
                    int_img[y, x + w // 2]
                    + int_img[y + h, x + w - 1]
                    - int_img[y, x + w]
                    - int_img[y + h, x + w // 2 - 1]
                )
            features.append(black - white)

    return features


def compact_haar_features3(img):
    int_img = integral_image(img)
    h, w = 7, 7  # Size of the grid cell
    features = []

    for y in range(0, img.shape[0], h):
        for x in range(0, img.shape[1], w):
            # Horizontal edge feature
            white = (
                int_img[y + h // 2 - 1, x + w - 1]
                + int_img[y, x]
                - int_img[y, x + w - 1]
                - int_img[y + h // 2 - 1, x]
            )
            black = (
                int_img[y + h - 1, x + w - 1]
                + int_img[y + h // 2, x]
                - int_img[y + h // 2, x + w - 1]
                - int_img[y + h - 1, x]
            )
            features.append(black - white)

            # Vertical edge feature
            white = (
                int_img[y + h - 1, x + w // 2 - 1]
                + int_img[y, x]
                - int_img[y, x + w // 2 - 1]
                - int_img[y + h - 1, x]
            )
            black = (
                int_img[y + h - 1, x + w - 1]
                + int_img[y, x + w // 2]
                - int_img[y, x + w - 1]
                - int_img[y + h - 1, x + w // 2]
            )
            features.append(black - white)

    return features

In [41]:
haar_1 = compact_haar_features3(train_images_2d[0].reshape(28, 28))
print(haar_1)

[0, 0, 758, 532, 2056, 367, 633, -633, 0, 0, -1226, 2557, -598, -740, 0, 0, 0, 0, 799, 775, 1334, 282, 0, 0, -471, 1113, -2573, -220, -9, -9, 0, 0]


In [42]:
len_haar_features = len(haar_1)
print("length of a haar feature", len_haar_features)

length of a haar feature 32


In [43]:
# compute datasets

num_train = len(train_images)
num_test = len(test_images)

haar_train = np.zeros((num_train, len_haar_features))
haar_test = np.zeros((num_test, len_haar_features))

for i in range(num_train):
    haar_train[i] = compact_haar_features3(train_images_2d[i].reshape(28, 28))

for i in range(num_test):
    haar_test[i] = compact_haar_features3(test_images_2d[i].reshape(28, 28))

In [44]:
train_features = haar_train
test_features = haar_test

In [45]:
from sklearn.tree import DecisionTreeClassifier

# Create and train a decision tree classifier
clf = DecisionTreeClassifier(max_depth=10, random_state=0)
clf.fit(train_features, train_labels)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [46]:
import numpy as np

num_test_samples = len(test_features)
python_predictions = clf.predict(test_features)
python_accuracy = np.sum(python_predictions == test_labels) / num_test_samples
print(f"Python accuracy: {100*python_accuracy} %")

Python accuracy: 77.17 %


In [47]:
print("classifier depth", clf.get_depth())

classifier depth 10


## Try out an SVM

In [48]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

In [49]:
from sklearn.svm import SVC

clf = SVC(kernel="rbf", random_state=0, C=1000)
clf.fit(train_features_normalized, train_labels)

SVC(C=1000, random_state=0)

In [50]:
num_test_samples = len(test_features)
python_predictions = clf.predict(test_features_normalized)
python_accuracy = np.sum(python_predictions == test_labels) / num_test_samples
print(f"Python accuracy: {100*python_accuracy} %")

print("Number of support vectors", clf.n_support_)
print("Total number of support vectors", sum(clf.n_support_))

Python accuracy: 92.22 %
Number of support vectors [ 778  597 1474 1683 1232 1689  933 1276 1737 1543]
Total number of support vectors 12942
